You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [5]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [6]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [7]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [8]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [9]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [10]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [11]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [12]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [13]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [14]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [15]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:
Content Plan Document:

Title: Unveiling the Latest Trends and Key Players in Artificial Intelligence

Introduction:
- Brief overview of artificial intelligence and its significance in today's technological landscape
- Mention of the rapid advancements and innovations in AI technology
- Preview of key trends and key players in the AI industry

Key Points:
1. Latest Trends in Artificial Intelligence:
   a. Machine Learning: Applications and advancements
   b. Natural Language Processing: Impact on communication

I now can give a great answer

Final Answer:
# Unveiling the Latest Trends and Key Players in Artificial Intelligence

## Introduction
Artificial Intelligence (AI) has become a cornerstone of today's technological landscape, revolutionizing industries and reshaping the way we interact with technology. The rapid advancements in AI have propelled innovation to new heights, leading to groundbreaking developments in various sectors. In this blog post, we will delve into the latest trends and key players in the AI industry, exploring the cutting-edge technologies shaping the future of AI.

## Latest Trends in Artificial Intelligence
### Machine Learning
Machine learning stands at the forefront of AI innovation, with applications ranging from personalized recommendations to autonomous vehicles. The continuous advancements in machine learning algorithms have enabled AI systems to learn from data and improve their performance over time, leading to more efficient and accurate outcomes in variou

- Display the results of your execution as markdown in the notebook.

In [16]:
from IPython.display import Markdown
Markdown(result)

# Unveiling the Latest Trends and Key Players in Artificial Intelligence

## Introduction
Artificial Intelligence (AI) has become a cornerstone of today's technological landscape, revolutionizing industries and reshaping the way we interact with technology. The rapid advancements in AI have propelled innovation to new heights, leading to groundbreaking developments in various sectors. In this blog post, we will delve into the latest trends and key players in the AI industry, exploring the cutting-edge technologies shaping the future of AI.

## Latest Trends in Artificial Intelligence
### Machine Learning
Machine learning stands at the forefront of AI innovation, with applications ranging from personalized recommendations to autonomous vehicles. The continuous advancements in machine learning algorithms have enabled AI systems to learn from data and improve their performance over time, leading to more efficient and accurate outcomes in various domains.

### Natural Language Processing
Natural Language Processing (NLP) has transformed the way we communicate with machines, enabling seamless interactions through voice assistants and chatbots. The impact of NLP in customer service has been particularly significant, enhancing user experiences and streamlining communication processes for businesses.

### Robotics
The integration of AI in robotics has paved the way for automation and efficiency in industries such as manufacturing and healthcare. AI-driven robots are capable of performing complex tasks with precision, leading to increased productivity and cost savings for organizations embracing robotic automation.

### Predictive Analytics
Predictive analytics harnesses the power of AI to make data-driven decisions and forecast future outcomes with accuracy. By analyzing vast amounts of data, AI algorithms can identify patterns and trends, empowering businesses to optimize operations and drive strategic decision-making based on predictive insights.

### AI Ethics
As AI technologies continue to evolve, the discussion on ethical considerations in AI development and usage has become increasingly prominent. Addressing issues such as bias, transparency, and accountability is crucial to ensuring the responsible and ethical deployment of AI systems in society.

## Key Players in Artificial Intelligence
### Google
Google remains a key player in the AI industry, driving innovation through projects like DeepMind and Google Brain. Their AI research and development efforts have led to groundbreaking discoveries in machine learning and neural networks, shaping the future of AI technologies.

### IBM
IBM's Watson AI platform has been instrumental in providing businesses and researchers with advanced AI capabilities for data analysis, cognitive computing, and natural language processing. Their contributions to AI research and applications have positioned them as a leader in the field.

### Amazon
Amazon leverages AI technologies to power products like Alexa and Amazon Web Services, offering cutting-edge solutions for voice recognition, natural language understanding, and cloud computing. Their commitment to AI innovation has solidified their position as a major player in the AI industry.

### Microsoft
Microsoft's Azure AI services and ongoing research in cognitive computing have positioned them as a frontrunner in AI technology development. Their contributions to AI-driven solutions for businesses and consumers have driven significant advancements in the field.

### Tesla
Tesla's focus on AI-driven autonomous vehicles and advancements in AI hardware has revolutionized the automotive industry. Their innovative approach to integrating AI technologies into electric vehicles has set new standards for safety, efficiency, and autonomous capabilities.

## Target Audience Analysis
Tech enthusiasts, professionals, and students interested in AI will find valuable insights in the latest trends and key players highlighted in this blog post. Business owners looking to implement AI solutions in their operations can gain valuable knowledge about the innovative AI technologies driving efficiency and automation. Researchers and academics studying artificial intelligence will benefit from the comprehensive overview of AI advancements and key players in the industry. Consumers curious about the impact of AI on daily life will discover how AI technologies are shaping the way we interact with technology and the world around us.

## Call to Action
As AI continues to evolve, it is essential for readers to stay updated on the latest trends and innovations in the AI industry. Engaging in discussions on AI ethics and responsible AI usage is crucial for shaping the future of AI technology in a positive and impactful way. For further reading and exploration of AI topics, readers are encouraged to explore resources provided by Forbes, MIT Technology Review, TechCrunch, Gartner, and OpenAI to stay informed and engaged with the latest developments in artificial intelligence.

In conclusion, this blog post aims to provide a comprehensive overview of the latest trends and key players in the artificial intelligence industry, catering to a diverse audience interested in the advancements and impact of AI technology. Stay informed, stay engaged, and be part of the ever-evolving landscape of artificial intelligence.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [18]:
topic = "Create a leetcode study plan for me. I want to specifically focus on dynamic programming. "
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Create a leetcode study plan for me. I want to specifically focus on dynamic programming. .
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now have all the necessary tools and information to create a comprehensive content plan on creating a Leetcode study plan focusing on dynamic programming. I am confident in my ability to provide a detailed outline, analyze the target audience, incorporate relevant SEO keywords, and include credible resources in my final answer.

Final Answer:
Content Plan Document

Title: Create a Leetcode Study Plan: Mastering Dynamic Programming

Introduction:
- Brief overview of the importance of dyn

I now can give a great answer

Final Answer:
# Create a Leetcode Study Plan: Mastering Dynamic Programming

Dynamic programming serves as a fundamental technique in algorithmic problem-solving, enabling individuals to tackle complex problems by breaking them down into smaller, more manageable subproblems. Platforms like Leetcode offer a valuable opportunity for individuals to hone their coding skills and deepen their understanding of dynamic programming concepts. By developing a structured study plan centered around mastering dynamic programming on Leetcode, individuals can engage in consistent practice and witness tangible improvement in their coding proficiency.

Understanding the core principles of dynamic programming is essential for effective problem-solving. This method involves dividing a problem into smaller subproblems and solving each subproblem only once while storing the results to prevent redundant computations. By leveraging dynamic programming techniques, individuals can

In [19]:
Markdown(result)

# Create a Leetcode Study Plan: Mastering Dynamic Programming

Dynamic programming serves as a fundamental technique in algorithmic problem-solving, enabling individuals to tackle complex problems by breaking them down into smaller, more manageable subproblems. Platforms like Leetcode offer a valuable opportunity for individuals to hone their coding skills and deepen their understanding of dynamic programming concepts. By developing a structured study plan centered around mastering dynamic programming on Leetcode, individuals can engage in consistent practice and witness tangible improvement in their coding proficiency.

Understanding the core principles of dynamic programming is essential for effective problem-solving. This method involves dividing a problem into smaller subproblems and solving each subproblem only once while storing the results to prevent redundant computations. By leveraging dynamic programming techniques, individuals can significantly enhance the efficiency of their algorithms, particularly when addressing optimization problems. Leetcode presents a diverse array of dynamic programming challenges, including popular problems like the Fibonacci sequence and the Longest Increasing Subsequence, providing ample opportunities for individuals to apply these principles in a practical context.

Mastering dynamic programming on Leetcode necessitates the development of a well-defined study plan. By establishing specific goals and outlining key learning objectives, individuals can structure their study plan in a manner that promotes organization and focus throughout their coding journey. Additionally, breaking down the study plan into manageable tasks and incorporating resources such as dynamic programming tutorials and practice problems on Leetcode can offer valuable insights and practice opportunities at varying levels of proficiency. Through diligent planning and regular self-assessment, individuals can monitor their progress, make necessary adjustments, and sustain continuous growth in their coding abilities.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).